# Imports e inicialización del servidor

In [1]:
import mysql.connector
import os, getpass, re
import gensim
import pandas as pd
import numpy as np
import cvxpy as cp
import tensorflow as tf
from mysql.connector import Error
from itertools import combinations
from ast import literal_eval
from collections import defaultdict
from scipy.spatial import distance
K = 10

In [2]:
# Definición de parámetros de conexión
connection_params = {
    'host': 'localhost',
    'user': 'cmescobar',
    'database': 'foodb',
    'password': getpass.getpass(prompt='Introduzca la contraseña: ')
}

try:
    connection = mysql.connector.connect(**connection_params)

    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()

except Error as e:
    print("Error while connecting to MySQL", e)

Introduzca la contraseña: ········
Connected to MySQL Server version  8.0.26


# Definición de la función objetivo customizada

In [3]:
def weighted_binary_crossentropy(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float64)
    y_pred = tf.cast(y_pred, tf.float64)
    bcross_ent = - 1 / len(y_true) * (y_true * tf.keras.backend.log(y_pred) + 
                                      (1 - y_true) * tf.keras.backend.log(1 - y_pred))
    weighted_bcross_ent = ((K - 1) * y_true + 1) * bcross_ent
    
    return tf.reduce_sum(weighted_bcross_ent)

In [4]:
weighted_binary_crossentropy(a, b)

NameError: name 'a' is not defined

In [5]:
a = np.array([0,0,0,0,0,1,0,0,0,0])
np.random.seed(0)
b = np.random.uniform(0, 1, 10)

print(a)
print(b)

print(- 1 / 10 * np.sum((a * np.log(b) + (1 - a) * np.log(1 - b) )))

bc = tf.keras.losses.BinaryCrossentropy(from_logits=False)
bc(a, b)

[0 0 0 0 0 1 0 0 0 0]
[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548  0.64589411
 0.43758721 0.891773   0.96366276 0.38344152]
1.1347955655799193


<tf.Tensor: shape=(), dtype=float64, numpy=1.134795069694519>

# Obteniendo la información de la tabla de interés

In [6]:
query = \
'''SELECT name, name_scientific, description, wikipedia_id
   FROM foods f;
'''

# Solicitud de la query
df = pd.read_sql(query, connection)

# Limpieza de datos

In [7]:
# Eliminando los alimentos con None en la descripción
df_clean = df.dropna(subset=['description'])
df_clean

,name,name_scientific,description,wikipedia_id
0,Angelica,Angelica keiskei,Angelica is a genus of about 60 species of tal...,Angelica
1,Savoy cabbage,Brassica oleracea var. sabauda,Savoy cabbage (Brassica oleracea convar. capit...,Savoy cabbage
2,Silver linden,Tilia argentea,Tilia tomentosa (Silver Lime in the UK and Sil...,Tilia tomentosa
3,Kiwi,Actinidia chinensis,"The kiwifruit, often shortened to kiwi in many...",Kiwifruit
4,Allium,Allium,Allium haematochiton is a species of wild onio...,Allium haematochiton
...,...,...,...,...
986,White bread,None,White bread typically refers to breads made fr...,None
987,Cape gooseberry,Physalis peruviana,"Physalis peruviana, a plant species of the gen...",None
988,Herbal tea,None,Herbal teas are the beverages made from the in...,None
989,Fish oil,None,Fish oil is oil derived from the tissues of oi...,None


# Definición de los datos de entrada y salida

In [8]:
X_data = df_clean['description']
Y_data = df_clean['name']

In [9]:
Y_labels = tf.keras.utils.to_categorical([i for i in range(len(Y_data))], 
                                         num_classes=None, dtype='int')

# Limpieza del texto

In [10]:
def depure_text(text):
    #Removing URLs with a regular expression
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    text = url_pattern.sub(r'', text)

    # Remove Emails
    text = re.sub('\S*@\S*\s?', '', text)

    # Remove new line characters
    text = re.sub('\s+', ' ', text)

    # Remove distracting single quotes
    text = re.sub("\'", "", text)
        
    return text


def sent_to_words(descriptions):
    '''Función que preprocesa el texto, y además elimina palabras muy cortas o 
    muy largas.
    
    Parameters
    ----------
    descriptions : list
        Lista de descripciones a revisar.
    '''
    for desc in descriptions:
        yield gensim.utils.simple_preprocess(depure_text(desc), deacc=True)
        
        
def detokenize(text):
    return nltk.tokenize.treebank.TreebankWordDetokenizer().detokenize(text)

In [11]:
# Obteniendo la lista de descripciones
X_data = list(sent_to_words(X_data))

# Definición del set de todas las palabras que se encuentran
# en las descripciones
bag_of_words = set()

for line in X_data:
    for i in line:
        bag_of_words.add(i)

In [12]:
len(bag_of_words)

11299

# Tokenization y Embeddings

In [13]:
# Definición de los parámetros de tokenization
max_words = 5000     # 10000
max_len = 200

# Definición del token
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)

# Ajustando el tokenizer a los textos
tokenizer.fit_on_texts(X_data)

# Una vez definido el token para cada palabra, se transforman los textos mediante cada token
sequences = tokenizer.texts_to_sequences(X_data)

# Dado que las secuencias no tienen el mismo largo, se paddea para obtener una matriz
X_token = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_len)
print(X_token.shape)
X_token

(950, 200)


array([[   0,    0,    0, ..., 3928,    2, 3929],
       [   0,    0,    0, ...,  969,    2, 2166],
       [   0,    0,    0, ...,    2,  341, 2517],
       ...,
       [   0,    0,    0, ..., 1555,  261, 1669],
       [   0,    0,    0, ...,   14,    3, 3736],
       [   0,    0,    0, ...,  567,    5,    5]])

# RNN construction

In [83]:
model = tf.keras.Sequential()
# model.add(tf.keras.layers.Masking(mask_value=0))
model.add(tf.keras.layers.Embedding(max_words, 64))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, dropout=0.5)))
# model.add(tf.keras.layers.Conv1D(20, 6, activation='relu',
#                         kernel_regularizer=tf.keras.regularizers.l1_l2(l1=2e-3, l2=2e-3),
#                         bias_regularizer=tf.keras.regularizers.l2(2e-3)))
# model.add(tf.keras.layers.MaxPooling1D(5))
# model.add(tf.keras.layers.Conv1D(20, 6, activation='relu',
#                         kernel_regularizer=tf.keras.regularizers.l1_l2(l1=2e-3, l2=2e-3),
#                         bias_regularizer=tf.keras.regularizers.l2(2e-3)))
# model.add(tf.keras.layers.GlobalMaxPooling1D())
model.add(tf.keras.layers.Dense(32, activation='relu'))
# model.add(tf.keras.layers.Dense(64, activation='relu'))
# QUizás es mejor usar sigmoide en vez de softmax, porque cada una es independiente

model.add(tf.keras.layers.Dense(len(Y_data), activation='softmax'))
# model.add(tf.keras.layers.Dense(len(Y_data), activation='sigmoid'))

# Compilando
# model.compile(optimizer='rmsprop', loss=weighted_binary_crossentropy, metrics=['accuracy'])
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
# model.compile(optimizer='rmsprop', loss='MSE', metrics=['accuracy'])

#Implementing model checkpoins to save the best metric and do not lose it on training.
# checkpoint = tf.keras.callbacks.ModelCheckpoint("best_model1.hdf5", monitor='val_accuracy', 
#                                                 verbose=1, save_best_only=True, 
#                                                 mode='auto', period=1,save_weights_only=False)

# Implementando un EarlyStopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

# Entrenando
history = model.fit(X_token, Y_labels, epochs=250, callbacks=[early_stopping], batch_size=32)

Epoch 1/250
30/30 [==============================] - 0s 16ms/step - loss: 0.0083 - accuracy: 0.0000e+00
Epoch 2/250
30/30 [==============================] - 0s 15ms/step - loss: 0.0083 - accuracy: 0.0000e+00
Epoch 3/250
30/30 [==============================] - 0s 15ms/step - loss: 0.0083 - accuracy: 0.0000e+00
Epoch 4/250
30/30 [==============================] - 0s 15ms/step - loss: 0.0083 - accuracy: 0.0011
Epoch 5/250
30/30 [==============================] - 0s 15ms/step - loss: 0.0083 - accuracy: 0.0000e+00
Epoch 6/250
30/30 [==============================] - 0s 15ms/step - loss: 0.0083 - accuracy: 0.0000e+00
Epoch 7/250
30/30 [==============================] - 0s 15ms/step - loss: 0.0083 - accuracy: 0.0000e+00
Epoch 8/250
30/30 [==============================] - 0s 15ms/step - loss: 0.0083 - accuracy: 0.0032
Epoch 9/250
30/30 [==============================] - 0s 15ms/step - loss: 0.0083 - accuracy: 0.0011
Epoch 10/250
30/30 [==============================] - 0s 15ms/step - loss: 0

30/30 [==============================] - 0s 15ms/step - loss: 0.0023 - accuracy: 0.6337
Epoch 82/250
30/30 [==============================] - 0s 15ms/step - loss: 0.0024 - accuracy: 0.6316
Epoch 83/250
30/30 [==============================] - 0s 15ms/step - loss: 0.0022 - accuracy: 0.6579
Epoch 84/250
30/30 [==============================] - 0s 15ms/step - loss: 0.0022 - accuracy: 0.6421
Epoch 85/250
30/30 [==============================] - 0s 15ms/step - loss: 0.0022 - accuracy: 0.6821
Epoch 86/250
30/30 [==============================] - 0s 15ms/step - loss: 0.0022 - accuracy: 0.6674
Epoch 87/250
30/30 [==============================] - 0s 15ms/step - loss: 0.0023 - accuracy: 0.6863
Epoch 88/250
30/30 [==============================] - 0s 15ms/step - loss: 0.0022 - accuracy: 0.6779
Epoch 89/250
30/30 [==============================] - 0s 15ms/step - loss: 0.0022 - accuracy: 0.6947
Epoch 90/250
30/30 [==============================] - 0s 15ms/step - loss: 0.0022 - accuracy: 0.7105
Epo

In [16]:
# history = model.fit(X_token, Y_labels, epochs=100, callbacks=[early_stopping])

# RNN prediction

In [84]:
text_to = ['I am a citric fruit'.split(' ')]
print(text_to)

# Una vez definido el token para cada palabra, se transforman los textos mediante cada token
text_to = tokenizer.texts_to_sequences(text_to)
print(text_to)

# Dado que las secuencias no tienen el mismo largo, se paddea para obtener una matriz
text_to = tf.keras.preprocessing.sequence.pad_sequences(text_to, maxlen=max_len)

[['I', 'am', 'a', 'citric', 'fruit']]
[[4036, 22]]


In [85]:
text_to

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

In [86]:
a = model.predict(text_to)

In [87]:
a.shape

(1, 950)

In [88]:
b = [(num, i) for num, i in enumerate(a[0])]
b.sort(key=lambda x: x[1], reverse=True)

In [89]:
b

[(854, 0.19641523),
 (881, 0.123817064),
 (851, 0.11977711),
 (855, 0.11374322),
 (852, 0.110576436),
 (853, 0.10568116),
 (885, 0.07485688),
 (507, 0.03676902),
 (886, 0.030280465),
 (263, 0.0153230475),
 (754, 0.009063104),
 (442, 0.0077666473),
 (136, 0.007391167),
 (252, 0.006594637),
 (190, 0.0065542585),
 (130, 0.0060435045),
 (497, 0.002480836),
 (391, 0.0021971276),
 (871, 0.0017593907),
 (890, 0.0015576164),
 (71, 0.001406116),
 (872, 0.001322961),
 (48, 0.0013019834),
 (447, 0.0012670193),
 (752, 0.0012445147),
 (315, 0.0010873915),
 (892, 0.001030048),
 (258, 0.0009407718),
 (627, 0.0007968951),
 (943, 0.00074752927),
 (917, 0.0007003233),
 (866, 0.00063524814),
 (817, 0.00062035746),
 (676, 0.0005634559),
 (560, 0.0005304175),
 (269, 0.0005278298),
 (901, 0.00041565343),
 (914, 0.00041553928),
 (227, 0.0003950911),
 (921, 0.0003165528),
 (14, 0.000301937),
 (606, 0.00029423015),
 (336, 0.0002659162),
 (949, 0.0002578194),
 (203, 0.000245666),
 (163, 0.00024152538),
 (525, 0

In [90]:
for i in b:
    print(Y_data.tolist()[i[0]], round(i[1] * 100, 2), '%')

Albizia gummifera 19.64 %
Dessert wine 12.38 %
Castanospermum australe 11.98 %
Mundu 11.37 %
Gentiana lutea 11.06 %
Juniperus communis 10.57 %
Red grape juice 7.49 %
Clupeinae (Herring, Sardine, Sprat) 3.68 %
White grape juice 3.03 %
Biscuit 1.53 %
Cooking oil 0.91 %
Atlantic salmon 0.78 %
Anise 0.74 %
Italian oregano 0.66 %
American cranberry 0.66 %
Parsley 0.6 %
Yautia 0.25 %
Malabar spinach 0.22 %
Rapeseed oil 0.18 %
Mikan 0.16 %
Wild carrot 0.14 %
Soybean oil 0.13 %
Chicory 0.13 %
Spanish mackerel 0.13 %
Ucuhuba 0.12 %
Chia 0.11 %
Plain cream cheese 0.1 %
Skunk currant 0.09 %
Dragée 0.08 %
Hawthorn 0.07 %
Clementine 0.07 %
Green tea 0.06 %
Alcoholic beverages 0.06 %
Zwieback 0.06 %
Crab 0.05 %
Acorn 0.05 %
Cottonseed oil 0.04 %
Greek feta cheese 0.04 %
Pepper (C. baccatum) 0.04 %
Green onion 0.03 %
Wild celery 0.03 %
Bean 0.03 %
Devilfish 0.03 %
Taco shell 0.03 %
Corn 0.02 %
Sorrel 0.02 %
Alaska wild rhubarb 0.02 %
Greater sturgeon 0.02 %
Green lentil 0.02 %
Mexican oregano 0.02 %


Northern bluefin tuna 0.0 %
Borage 0.0 %
Adobo 0.0 %
Other candy 0.0 %
Pepper (C. pubescens) 0.0 %
Lichee 0.0 %
Black walnut 0.0 %
Pan dulce 0.0 %
Molasses 0.0 %
Rosemary 0.0 %
Chervil 0.0 %
Domestic pig 0.0 %
Vaccinium (Blueberry, Cranberry, Huckleberry) 0.0 %
Popcorn 0.0 %
Pacific cod 0.0 %
Dumpling 0.0 %
Leavening agent 0.0 %
Pine nut 0.0 %
Catfish 0.0 %
Lemon sole 0.0 %
Muskmelon 0.0 %
Cold cut 0.0 %
Coalfish pollock 0.0 %
Cattle (Beef, Veal) 0.0 %
Macadamia nut (M. tetraphylla) 0.0 %
Cumin 0.0 %
Common grape 0.0 %
Kelp 0.0 %
Cusk 0.0 %
Soy bean 0.0 %
Northern pike 0.0 %
Japanese chestnut 0.0 %
Nougat 0.0 %
Roselle 0.0 %
Allspice 0.0 %
Macroalgae 0.0 %
Marine mussel 0.0 %
Common bean 0.0 %
Oil-seed Camellia 0.0 %
Red raspberry 0.0 %
Hibiscus tea 0.0 %
Albacore tuna 0.0 %
Cubanelle pepper 0.0 %
Chocolate spread 0.0 %
Red beetroot 0.0 %
Other sandwich 0.0 %
Dulce de leche 0.0 %
Moth bean 0.0 %
Hickory nut 0.0 %
Norway lobster 0.0 %
Hyacinth bean 0.0 %
Tinda 0.0 %
Wild rice 0.0 %
Caro